# Emplifi API Testing - BAYER Listening Query

This notebook provides simple testing for the Emplifi API using the BAYER listening query. It demonstrates how to use our MCP tools to fetch posts and verify API functionality.

**Target Query**: BAYER (ID: LNQ_1140092_66fe2dcd3e9eb298096e8db3)

## Import Required Libraries

Import necessary libraries including our custom Emplifi API modules, pandas for data analysis, and datetime for date range testing.

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import json
from datetime import datetime, timedelta, timezone

# Add the src directory to the path so we can import our modules
notebook_dir = Path().resolve()
src_dir = notebook_dir.parent / "src"
sys.path.insert(0, str(src_dir))

# Import our Emplifi tools
from mcp_server.tools.emplifi_tools import (
    list_listening_queries,
    fetch_listening_posts,
    get_recent_posts
)

print("✅ Libraries imported successfully")
print(f"📁 Working from: {notebook_dir}")
print(f"📦 Source directory: {src_dir}")

# Test constants for BAYER query
TARGET_QUERY_ID = "LNQ_1140092_66fe2dcd3e9eb298096e8db3"
TARGET_QUERY_NAME = "BAYER"

print(f"\n🎯 Target Query: {TARGET_QUERY_NAME}")
print(f"   ID: {TARGET_QUERY_ID}")

✅ Libraries imported successfully
📁 Working from: /home/gferreir/mines/mcp/mcp_template/notebooks
📦 Source directory: /home/gferreir/mines/mcp/mcp_template/src

🎯 Target Query: BAYER
   ID: LNQ_1140092_66fe2dcd3e9eb298096e8db3


## Test Environment Setup

Verify that our environment variables are configured correctly for API access.

In [ ]:
# Test environment variables
emplifi_token = os.environ.get("EMPLIFI_TOKEN")
emplifi_secret = os.environ.get("EMPLIFI_SECRET")

print("🔐 Environment Check:")
print(f"   EMPLIFI_TOKEN: {'✅ Set' if emplifi_token else '❌ Not set'}")
print(f"   EMPLIFI_SECRET: {'✅ Set' if emplifi_secret else '❌ Not set'}")

if emplifi_token and emplifi_secret:
    print("\n✅ Environment configured correctly")
else:
    print("\n❌ Missing environment variables")
    print("💡 Make sure your .env file contains EMPLIFI_TOKEN and EMPLIFI_SECRET")

## Verify BAYER Query Exists

Check that our target BAYER query is available and active in the account.

In [ ]:
# Test: List all queries and find our BAYER query
try:
    print("🔍 Fetching all listening queries...")
    queries = await list_listening_queries()
    
    print(f"📊 Total queries found: {len(queries)}")
    
    # Find our specific BAYER query
    bayer_query = None
    for query in queries:
        if query.id == TARGET_QUERY_ID:
            bayer_query = query
            break
    
    if bayer_query:
        print(f"\n✅ Found BAYER query!")
        print(f"   ID: {bayer_query.id}")
        print(f"   Name: '{bayer_query.name}'")
        print(f"   Status: {bayer_query.status}")
        
        if bayer_query.status == "Running":
            print("✅ Query is active and running")
        else:
            print(f"⚠️  Query status: {bayer_query.status}")
    else:
        print(f"❌ BAYER query not found!")
        print(f"   Looking for ID: {TARGET_QUERY_ID}")
        
        # Show first few queries for reference
        print("\n📝 First 5 available queries:")
        for i, q in enumerate(queries[:5]):
            print(f"   {i+1}. {q.name} ({q.id}) - {q.status}")
        
except Exception as e:
    print(f"❌ Error fetching queries: {e}")
    import traceback
    traceback.print_exc()

## Test Recent Posts Retrieval

Test the `get_recent_posts` function to fetch recent posts from the BAYER query.

In [ ]:
# Test: Get recent posts from BAYER query
try:
    print(f"🔍 Testing get_recent_posts for BAYER query...")
    print(f"   Query ID: {TARGET_QUERY_ID}")
    
    # Test with last 7 days
    recent_posts = await get_recent_posts(
        query_id=TARGET_QUERY_ID,
        days_back=7,
        limit=5
    )
    
    print(f"\n📊 Results:")
    print(f"   Found {len(recent_posts)} posts from last 7 days")
    
    if recent_posts:
        print("\n📝 Sample posts:")
        for i, post in enumerate(recent_posts[:3], 1):
            print(f"   {i}. Platform: {post.platform}")
            print(f"      Created: {post.created_time}")
            print(f"      Interactions: {post.interactions}")
            print(f"      Message preview: {post.message[:100]}...")
            print(f"      Sentiment: {post.sentiment}")
            print()
    else:
        print("   No posts found in the last 7 days")
        
        # Try a longer period
        print("\n🔍 Trying last 30 days...")
        older_posts = await get_recent_posts(
            query_id=TARGET_QUERY_ID,
            days_back=30,
            limit=5
        )
        print(f"   Found {len(older_posts)} posts from last 30 days")
        
except Exception as e:
    print(f"❌ Error fetching recent posts: {e}")
    import traceback
    traceback.print_exc()

## Test Date Range Posts Retrieval

Test the `fetch_listening_posts` function with specific date ranges.

In [ ]:
# Test: Fetch posts for specific date range
try:
    print(f"🔍 Testing fetch_listening_posts with date range...")
    
    # Test with a recent week that we know has data
    start_date = "2025-08-05"
    end_date = "2025-08-12"
    
    print(f"   Date range: {start_date} to {end_date}")
    print(f"   Query ID: {TARGET_QUERY_ID}")
    
    posts = await fetch_listening_posts(
        query_ids=[TARGET_QUERY_ID],
        date_start=start_date,
        date_end=end_date,
        limit=10
    )
    
    print(f"\n📊 Results:")
    print(f"   Found {len(posts)} posts in date range")
    
    if posts:
        print("\n📈 Post details:")
        for i, post in enumerate(posts[:5], 1):
            print(f"   {i}. {post.platform.upper()} - {post.created_time}")
            print(f"      Interactions: {post.interactions:,}")
            print(f"      Sentiment: {post.sentiment}")
            print(f"      Message: {post.message[:80]}...")
            print()
            
        # Show interaction distribution
        interactions = [p.interactions for p in posts if p.interactions]
        if interactions:
            print(f"📊 Interaction stats:")
            print(f"   Total posts with interactions: {len(interactions)}")
            print(f"   Highest interactions: {max(interactions):,}")
            print(f"   Average interactions: {sum(interactions)/len(interactions):.1f}")
    else:
        print("   No posts found in this date range")
        
except Exception as e:
    print(f"❌ Error fetching posts: {e}")
    import traceback
    traceback.print_exc()

## Data Analysis

Convert posts to a pandas DataFrame for easy analysis and visualization.

In [ ]:
# Convert posts to DataFrame for analysis
if 'posts' in locals() and posts:
    # Create DataFrame from posts
    df_posts = pd.DataFrame([{
        "id": post.id,
        "created_time": post.created_time,
        "platform": post.platform,
        "author_name": post.author.get("name", "Unknown") if post.author else "Unknown",
        "message_length": len(post.message) if post.message else 0,
        "message_preview": post.message[:100] + "..." if post.message and len(post.message) > 100 else post.message,
        "sentiment": post.sentiment,
        "interactions": post.interactions or 0,
        "has_url": bool(post.url)
    } for post in posts])
    
    print(f"📊 DataFrame created with {len(df_posts)} posts")
    print(f"\n📈 Summary statistics:")
    print(f"   Platforms: {df_posts['platform'].value_counts().to_dict()}")
    print(f"   Sentiments: {df_posts['sentiment'].value_counts().to_dict()}")
    print(f"   Avg interactions: {df_posts['interactions'].mean():.1f}")
    print(f"   Max interactions: {df_posts['interactions'].max():,}")
    
    # Display the DataFrame
    print(f"\n📝 Posts data:")
    display(df_posts[['platform', 'created_time', 'sentiment', 'interactions', 'message_preview']])
    
else:
    print("❌ No posts data available to analyze")
    print("   Make sure to run the previous cells to fetch posts first")

In [ ]:
# Quick test summary
print("🧪 EMPLIFI API TEST SUMMARY")
print("=" * 40)

try:
    # Test 1: List queries
    query_count = len(await list_listening_queries())
    print(f"✅ list_listening_queries: {query_count} queries found")
except Exception as e:
    print(f"❌ list_listening_queries: {e}")

try:
    # Test 2: Recent posts
    recent = await get_recent_posts(TARGET_QUERY_ID, days_back=7, limit=3)
    print(f"✅ get_recent_posts: {len(recent)} posts (last 7 days)")
except Exception as e:
    print(f"❌ get_recent_posts: {e}")

try:
    # Test 3: Date range posts
    range_posts = await fetch_listening_posts(
        query_ids=[TARGET_QUERY_ID],
        date_start="2025-08-05",
        date_end="2025-08-12",
        limit=5
    )
    print(f"✅ fetch_listening_posts: {len(range_posts)} posts (Aug 5-12)")
except Exception as e:
    print(f"❌ fetch_listening_posts: {e}")

print("\n🎯 All tests completed!")